In [2]:
import requests
from requests.exceptions import RequestException
import pandas as pd
import datetime
from datetime import datetime, timedelta



url = "https://api.data.gov.sg/v1/environment/psi"

# Function to fetch PSI data for a given date
def fetch_psi(date):
    try:
        r = requests.get(url, params={'date': date})
        r.raise_for_status()  # Raises HTTPError for bad responses (4xx, 5xx)
        return r.json()
    except RequestException as e:
        print(f"Error fetching data for {date}: {e}")
        return {}

# Function to fetch data between start and end date
def fetch_psi_range(start, end):
    d = datetime.strptime(start, '%Y-%m-%d')
    e = datetime.strptime(end, '%Y-%m-%d')
    data = []
    while d <= e:
        day_data = fetch_psi(d.strftime('%Y-%m-%d'))
        if 'items' in day_data and day_data['items']:
            data.extend(day_data['items'])
        d += timedelta(days=1)
    return data, day_data.get('region_metadata', [])

# Period of Extraction , only available from 2016 Feb onwards
start_date = '2016-02-01'
end_date = '2024-01-01'
psi_data, region_metadata = fetch_psi_range(start_date, end_date)

# Extract readings and structure them by region and pollutant type, including timestamps
readings_list = []

for item in psi_data:
    timestamp = item['timestamp']  # Extract timestamp
    update_timestamp = item['update_timestamp']  # Extract update_timestamp

    # Iterate through pollutant readings
    for no, region_data in item['readings'].items():
        for region, value in region_data.items():
            readings_list.append({
                'region': region,
                'no': no,
                'value': value,
                'timestamp': timestamp,
                'update_timestamp': update_timestamp
            })

# Convert list of readings to DataFrame
df_readings = pd.DataFrame(readings_list)

# Pivot the DataFrame to get pollutant types as columns
df_psi = df_readings.pivot_table(index=['region', 'timestamp', 'update_timestamp'],
                                     columns='no',
                                     values='value').reset_index()

# Create DataFrame for region metadata
df_region_metadata = pd.DataFrame(region_metadata)

display(df_psi)
display(df_region_metadata)

# Save region data
df_region_metadata.to_csv("df_region_metadata.csv", index=False)

no,region,timestamp,update_timestamp,co_eight_hour_max,co_sub_index,no2_one_hour_max,o3_eight_hour_max,o3_sub_index,pm10_sub_index,pm10_twenty_four_hourly,pm25_sub_index,pm25_twenty_four_hourly,psi_three_hourly,psi_twenty_four_hourly,so2_sub_index,so2_twenty_four_hourly
0,central,2016-02-07T18:00:00+08:00,,0.45,5.0,10.0,59.0,25.0,21.0,21.0,47.0,11.0,57.0,47.0,2.0,3.0
1,central,2016-02-08T23:00:00+08:00,,0.42,4.0,15.0,49.0,21.0,34.0,34.0,57.0,17.0,58.0,57.0,2.0,4.0
2,central,2016-02-09T01:00:00+08:00,,0.42,4.0,8.0,44.0,19.0,34.0,34.0,57.0,17.0,59.0,57.0,2.0,4.0
3,central,2016-02-09T02:00:00+08:00,,0.42,4.0,7.0,43.0,18.0,34.0,34.0,56.0,17.0,59.0,56.0,2.0,3.0
4,central,2016-02-09T03:00:00+08:00,,0.42,4.0,10.0,42.0,18.0,33.0,33.0,56.0,17.0,61.0,56.0,2.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378058,west,2024-01-01T19:00:00+08:00,2024-01-01T19:00:41+08:00,1.00,6.0,23.0,19.0,8.0,17.0,17.0,31.0,7.0,NaN,31.0,1.0,2.0
378059,west,2024-01-01T20:00:00+08:00,2024-01-01T20:00:41+08:00,1.00,7.0,28.0,17.0,7.0,18.0,18.0,32.0,8.0,NaN,32.0,1.0,2.0
378060,west,2024-01-01T21:00:00+08:00,2024-01-01T21:00:41+08:00,1.00,7.0,33.0,14.0,6.0,18.0,18.0,32.0,8.0,NaN,32.0,1.0,2.0
378061,west,2024-01-01T22:00:00+08:00,2024-01-01T22:00:42+08:00,1.00,7.0,23.0,12.0,5.0,18.0,18.0,33.0,8.0,NaN,33.0,1.0,2.0


,name,label_location
0,west,"{'latitude': 1.35735, 'longitude': 103.7}"
1,east,"{'latitude': 1.35735, 'longitude': 103.94}"
2,central,"{'latitude': 1.35735, 'longitude': 103.82}"
3,south,"{'latitude': 1.29587, 'longitude': 103.82}"
4,north,"{'latitude': 1.41803, 'longitude': 103.82}"


In [11]:
df_psi['year'] = pd.to_datetime(df_psi['timestamp']).dt.year

# Drop all sub_index columns
to_drop = ['update_timestamp','timestamp']

for i in df_psi.columns:
    if 'sub_index' in i:
        to_drop.append(i)

df_psi = df_psi.drop(columns=to_drop)
df_psi.to_csv('sg_air_quality_w_area_clean.csv',index=False)
df_psi.head(5)


,region,co_eight_hour_max,no2_one_hour_max,o3_eight_hour_max,pm10_twenty_four_hourly,pm25_twenty_four_hourly,psi_three_hourly,psi_twenty_four_hourly,so2_twenty_four_hourly,year
0,central,0.45,10.0,59.0,21.0,11.0,57.0,47.0,3.0,2016
1,central,0.42,15.0,49.0,34.0,17.0,58.0,57.0,4.0,2016
2,central,0.42,8.0,44.0,34.0,17.0,59.0,57.0,4.0,2016
3,central,0.42,7.0,43.0,34.0,17.0,59.0,56.0,3.0,2016
4,central,0.42,10.0,42.0,33.0,17.0,61.0,56.0,3.0,2016
